In [ ]:
from tqdm import tqdm
import numpy as np
import cv2
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import *
from keras.applications.efficientnet import *
from keras.preprocessing.image import ImageDataGenerator
from models import *
import os
import warnings

In [ ]:
warnings.filterwarnings('ignore')
plt.rcParams["figure.dpi"] = 120
image_dimension = (224, 224)
epochs=100
batch_size = 32
epoch_index = [i for i in range(1, epochs+1)]

#### Check whether GPU is available

In [ ]:
tf.config.list_physical_devices('GPU')

#### Load Data, Label for prediction (currently use one dataset (Brain MRI Data2))

In [ ]:
labels = {"glioma_tumor":1, "meningioma_tumor":2, "no_tumor":0, "pituitary_tumor":3}
dir_name = "MRI_DATA"
image_list = list()
truth_list = list()

for label, index in labels.items():
    path = os.path.join("Brain_MRI_data2", dir_name, label)
    images = os.listdir(path)
    for image in tqdm(images):
        current_image = cv2.imread(os.path.join(path, image))
        current_image = cv2.resize(current_image, image_dimension)
        image_list.append(current_image)
        truth_list.append(index)

features = np.array(image_list)/255.0 # rescale
labels = tf.keras.utils.to_categorical(truth_list)

In [ ]:
print("image dimension: {}, label dimension: {}".format(features.shape, labels.shape))

#### Define model saving function

In [ ]:
def save_model(model_name):
    checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join("trained_model_weights", model_name),
                                 monitor='val_accuracy',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='max')
    return [checkpoint]

#### Split dataset into train, validation and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42, shuffle=True)
train_generator = ImageDataGenerator(validation_split=0.25)
test_generator = ImageDataGenerator()

train_df = train_generator.flow(X_train, y_train, batch_size=batch_size, subset="training")
validation_df = train_generator.flow(X_train, y_train, batch_size=batch_size, subset="validation")
test_df = train_generator.flow(X_test, y_test, batch_size=batch_size)

#### Use a simple CNN

In [ ]:
simple_cnn_path = "simple_cnn.hdf5"
model_simple_cnn = simple_cnn()
model_simple_cnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_simple_cnn.summary()

In [ ]:
simple_cnn_history = model_simple_cnn.fit(train_df,
                                          epochs=epochs,
                                          verbose=1,
                                          validation_data=validation_df,
                                          callbacks=save_model(simple_cnn_path))

In [ ]:
model_simple_cnn.load_weights(os.path.join("trained_model_weights", simple_cnn_path))
model_simple_cnn.evaluate(test_df)

In [ ]:
simple_cnn_result = pd.DataFrame({"Accuracy": simple_cnn_history.history["val_accuracy"],
                                  "Loss": simple_cnn_history.history["val_loss"]}, index=epoch_index)
simple_cnn_result.to_csv("Simple_CNN_result.csv")
ax_simple_cnn = simple_cnn_result.plot.line(figsize=(16, 9))
ax_simple_cnn.set_xlabel("Epoch")
ax_simple_cnn.set_ylabel("Score")
ax_simple_cnn.set_title("Simple CNN Model")

#### Using Resnet50

In [ ]:
model_resnet_path = "resnet50.hdf5"
model_resnet = resnet_50()
model_resnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_resnet.summary()

In [ ]:
model_resnet_history = model_resnet.fit(train_df,
                 epochs=epochs,
                 verbose=1,
                 validation_data=validation_df,
                 callbacks=save_model(model_resnet_path))

In [ ]:
epoch_index = [i for i in range(1, epochs+1)]
resnet_result = pd.DataFrame({"Accuracy": model_resnet_history.history["val_accuracy"],
                                  "Loss": model_resnet_history.history["val_loss"]}, index=epoch_index)
resnet_result.to_csv("resnet_result.csv")
ax_resnet = resnet_result.plot.line(figsize=(16, 9))
ax_resnet.set_xlabel("Epoch")
ax_resnet.set_ylabel("Score")
ax_resnet.set_title("Resnet50 Model")

In [ ]:
model_resnet.load_weights(os.path.join("trained_model_weights", model_resnet_path))
model_resnet.evaluate(test_df)

#### Using VGG-16

In [ ]:
model_vgg16_path = "vgg16.hdf5"
model_vgg16 = vgg_16()
model_vgg16.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
model_vgg16.summary()

In [ ]:
model_vgg16_history = model_vgg16.fit(train_df,
                                      verbose=1,
                                      epochs=epochs,
                                      validation_data=validation_df,
                                      callbacks=save_model(model_vgg16_path))

In [ ]:
vgg16_result = pd.DataFrame({"Accuracy": model_vgg16_history.history["val_accuracy"],
                             "Loss": model_vgg16_history.history["val_loss"]}, index=epoch_index)
vgg16_result.to_csv("vgg16_result.csv")
ax_vgg16 = vgg16_result.plot.line(figsize=(16, 9))
ax_vgg16.set_xlabel("Epoch")
ax_vgg16.set_ylabel("Score")
ax_vgg16.set_title("VGG16 Model")

In [ ]:
model_vgg16.load_weights(os.path.join("trained_model_weights", model_vgg16_path))
model_vgg16.evaluate(test_df)

#### Using VGG-19

In [ ]:
model_vgg19_path = "vgg19.hdf5"
model_vgg19 = vgg_19()
model_vgg19.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
model_vgg19.summary()

In [ ]:
model_vgg19_history = model_vgg19.fit(train_df,
                                      verbose=1,
                                      epochs=epochs,
                                      validation_data=validation_df,
                                      callbacks=save_model(model_vgg19_path))

In [ ]:
vgg19_result = pd.DataFrame({"Accuracy": model_vgg19_history.history["val_accuracy"],
                             "Loss": model_vgg19_history.history["val_loss"]}, index=epoch_index)
vgg19_result.to_csv("vgg19_result.csv")
ax_vgg19 = vgg19_result.plot.line(figsize=(16, 9))
ax_vgg19.set_xlabel("Epoch")
ax_vgg19.set_ylabel("Score")
ax_vgg19.set_title("VGG16 Model")

In [ ]:
model_vgg19.load_weights(os.path.join("trained_model_weights", model_vgg19_path))
model_vgg19.evaluate(test_df)

#### Using efficientnet B0

In [ ]:
model_efficientnetB0_path = "efficientnetB0.hdf5"
model_efficientnetB0 = efficientnet(EfficientNetB0())
model_efficientnetB0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
model_efficientnetB0.summary()

In [ ]:
model_efficientnetB0_history = model_efficientnetB0.fit(train_df,
                                      verbose=1,
                                      epochs=epochs,
                                      validation_data=validation_df,
                                      callbacks=save_model(model_efficientnetB0_path))

In [ ]:
efficientnetB0_result = pd.DataFrame({"Accuracy": model_efficientnetB0_history.history["val_accuracy"],
                             "Loss": model_efficientnetB0_history.history["val_loss"]}, index=epoch_index)
efficientnetB0_result.to_csv("efficientnetB3_result.csv")
ax_efficientnetB3 = efficientnetB0_result.plot.line(figsize=(16, 9))
ax_vgg19.set_xlabel("Epoch")
ax_vgg19.set_ylabel("Score")
ax_vgg19.set_title("efficientnetB3 Model")

In [ ]:
model_efficientnetB0.load_weights(os.path.join("trained_model_weights", model_efficientnetB0_path))
model_efficientnetB0.evaluate(test_df)

In [ ]:
model_accuracy_compare = pd.DataFrame({"simple_cnn": simple_cnn_history.history["val_accuracy"],
                                       "resnet50": model_resnet_history.history["val_accuracy"],
                                       "vgg16": model_vgg16_history.history["val_accuracy"],
                                       "vgg19": model_vgg19_history.history["val_accuracy"],
                                       "efficientnetB3": model_efficientnetB0_history.history["val_accuracy"]})

model_accuracy_compare.plot.line(figsize=(16, 9))


model_loss_compare = pd.DataFrame({"simple_cnn": simple_cnn_history.history["val_loss"],
                                   "resnet50": model_resnet_history.history["val_loss"],
                                   "vgg16": model_vgg16_history.history["val_loss"],
                                   "vgg19": model_vgg19_history.history["val_loss"],
                                   "efficientnetB3": model_efficientnetB0_history.history["val_loss"]})

model_loss_compare.plot.line(figsize=(16, 9))